# 📈 SK증권 장전 공시 정보 자동화 시스템

## 🎯 프로젝트 목적
SK증권 Passive영업팀에서 매일 장시작 전에 발송하는 **주요 증시 & 공시 정보**를 자동으로 수집하고 정리하는 시스템입니다.

## 🏗️ 시스템 아키텍처

### 1. 데이터 소스
- **한국거래소(KRX)**: 공시 정보, 투자경고/위험 종목
- **인베스팅닷컴**: 경제 일정 및 지표
- **KIND 전자공시시스템**: 상세 공시 내용

### 2. 핵심 기능 모듈

#### 📅 `get_previous_business_day()`
**목적**: 정확한 영업일 계산
```python
# 주말(토,일)과 한국 공휴일을 제외한 직전 영업일을 찾음
kr_holidays = holidays.KR()  # 한국 공휴일 라이브러리
while prev_day.weekday() >= 5 or prev_day in kr_holidays:
    prev_day -= timedelta(days=1)
```
**왜 이렇게?**: 증권시장은 영업일에만 열리므로, 정확한 비교 기준일이 필요

---

#### 📋 `get_공시(date)`
**목적**: KOSPI200/KOSDAQ150 지수 구성 종목의 공시만 필터링

**핵심 로직**:
```python
def is_krx_index_component(td_tag) -> bool:
    img_tags = td_tag.find_all('img')
    index_tags = [img.get('alt') for img in img_tags if img.has_attr('alt')]
    return 'KOSPI200' in index_tags or 'KOSDAQ150' in index_tags
```

**왜 이렇게?**:
- **선별적 정보 제공**: 수천 개 종목 중 주요 지수 구성 종목만 모니터링
- **효율성**: Passive 운용팀에서 관리하는 ETF의 기초 자산과 직접 관련
- **HTML 파싱**: KIND 사이트의 이미지 태그로 지수 구성 여부 판단

**데이터 수집 과정**:
1. POST 요청으로 공시 데이터 조회
2. BeautifulSoup으로 HTML 파싱
3. 각 행(`<tr>`)에서 시간, 회사명, 제목 추출
4. JavaScript `onclick` 이벤트에서 공시 ID 추출
5. 공시 상세 링크 생성

---

#### ⚠️ `get_투자경고종목()` & `get_투자위험종목()`
**목적**: 투자자 보호를 위한 위험 종목 모니터링

**특징**:
- 동일한 API 엔드포인트, 다른 `pagePath`와 `code` 사용
- JSON 응답에서 `DS1` 배열의 `kor_isu_nm` 필드 추출

**왜 분리?**: 
- 경고(과열) vs 위험(부실화) 단계별 구분 필요
- 각각 다른 규제 조치와 투자 제한 사항

---

#### 📊 `get_주요일정(week)`
**목적**: 시장에 영향을 줄 수 있는 경제 지표 일정 수집

**필터링 로직**:
```python
excluded_keywords = ['신규 실업수당청구건수', '주택판매', '원유재고', ...]
if any(keyword in title for keyword in excluded_keywords):
    continue
```

**왜 필터링?**:
- **노이즈 제거**: 시장 영향이 제한적인 지표 제외
- **핵심 지표 집중**: GDP, 물가, 금리 등 주요 지표만 포함
- **가독성**: 과도한 정보로 인한 혼란 방지

**시간 필터링**:
```python
if date < previous_business_day:
    continue  # 이미 지난 일정은 제외
```

---

## 🔄 실행 흐름

### 셀 2: 장전 문자 생성
```python
now = datetime.now()
date = f'{now.year}{now.month:02d}{now.day:02d}'

# 금요일이면 다음주 일정도 추가
if now.weekday() == 4:
    다음주_주요일정 = get_주요일정('nextWeek')
```

**왜 금요일 특별 처리?**: 주말 동안 투자자들이 다음주 일정을 미리 파악할 수 있도록

### 셀 3: 상세 공시 분석
```python
keywords = ["자기주식", "증자", "감자", "배당", "합병", ...]
pattern = '|'.join(keywords)  # 정규표현식 OR 패턴 생성
filtered_df = df[df['공시제목'].str.contains(pattern)]
```

**키워드 선정 기준**:
- **주가 영향도**: 직접적인 주가 변동 요인
- **지수 구성 변경**: ETF 운용에 영향을 주는 사건들
- **투자자 관심도**: 개별 투자자들이 주목하는 공시 유형

---

## 💡 설계 철학

### 1. **자동화 우선**
- 매일 반복되는 업무의 완전 자동화
- 사람의 개입 없이도 정확한 정보 수집

### 2. **선별적 정보 제공**
- 전체 공시가 아닌 핵심 정보만 필터링
- 과도한 정보로 인한 피로도 방지

### 3. **실시간성**
- 당일과 전영업일 공시 동시 모니터링
- 시장 개장 전 최신 정보 제공

### 4. **확장성**
- 모듈화된 함수 구조로 새로운 데이터 소스 추가 용이
- 필터링 조건 변경 시 최소한의 코드 수정

### 5. **안정성**
- 외부 API 의존성 최소화를 위한 다중 데이터 소스 활용
- 예외 처리를 통한 서비스 연속성 보장

In [1]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
import json
from datetime import datetime, timedelta
import holidays

def get_previous_business_day():
    """
    직전 영업일 구하기

    Returns:
        datetime.datetime: 직전 영업일.
    """

    kr_holidays = holidays.KR()
    today = datetime.today().date()

    prev_day = today - timedelta(days=1)
    while prev_day.weekday() >= 5 or prev_day in kr_holidays:
        prev_day -= timedelta(days=1)

    return prev_day

def get_주요지표변동():
    """
    주요지표변동 조회
    """
    pass
    
def get_공시(date: str) -> pd.DataFrame:
    """
    공시 조회

    Args:
        date (str): 조회할 날짜. 형식: 'YYYYMMDD'.

    Returns:
        pd.DataFrame: 데이터프레임. 컬럼: [시간, 회사명, 공시제목, 링크].
    """

    def is_krx_index_component(td_tag) -> bool:
        """
        해당 종목이 KOSPI200 또는 KOSDAQ150 지수 구성 종목인지 확인
        """
        img_tags = td_tag.find_all('img')
        index_tags = [img.get('alt') for img in img_tags if img.has_attr('alt')]
        return 'KOSPI200' in index_tags or 'KOSDAQ150' in index_tags

    url = "https://kind.krx.co.kr/disclosure/todaydisclosure.do"

    payload = {    
        'method': 'searchTodayDisclosureSub',
        'currentPageSize': '5000',
        'pageIndex': '1',
        'orderMode': '0',
        'orderStat': 'D',
        'marketType': '', 
        'forward': 'todaydisclosure_sub',
        'searchMode': '', 
        'searchCodeType': '', 
        'chose': 'S',
        'todayFlag': 'N',
        'repIsuSrtCd': '',
        'kosdaqSegment': '',
        'selDate': date,
    }

    res = requests.post(url, data=payload)
    soup = BeautifulSoup(res.text, 'html.parser')
    rows = soup.find_all('tr')

    results = []

    for tr in rows:
        tds = tr.find_all('td')

        if len(tds) < 3:
            continue

        if not is_krx_index_component(tds[1]):
            continue

        time = tds[0].text.strip()
        company = tds[1].text.strip()
        title = tds[2].text.strip()
        
        onclick = tds[2].find('a').get('onclick', '')
        match = re.search(r"openDisclsViewer\('(\d+)'", onclick)
        disclosure_id = match.group(1)
        link = f'https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno={disclosure_id}&docno=&viewerhost=&viewerport='
        
        results.append([time, company, title, link])


    columns = ["시간", "회사명", "공시제목", "링크"]
    return pd.DataFrame(results, columns=columns)

def get_투자경고종목(date: str) -> str:
    """
    투자경고종목 조회

    Args:
        date (str): 조회할 날짜. 형식: 'YYYYMMDD'.

    Returns:
        str: 투자경고종목.
    """

    url = "https://moc.krx.co.kr/contents/MOC/99/MOC99000001.jspx"

    payload = {
        "market_gubun": "ALL",
        "fromDate": date,
        "toDate": date,
        "pagePath": "/contents/SVL/M/03020200/MOC03020200.jsp",
        "code": "YmD69RMDlLdJjotrXv2qYKFB8V9nBNKG35UprgSdoRZ47Zz/MwrFp84JbilNWZo1b/Lug2VdJQm4mfONjCCfNjRUF6IYVXJgpkKZeyxQ/qt2Ziyg7PkzzcufP2hPFIkroP8zo2lM8Xbh1B+jlRJtIQpE7r9xTp2sNhIQX7myevyuq55WOC+DRARslzH9ldx+",
        "pageFirstCall": "Y"
    }

    res = requests.post(url, data=payload)
    data = json.loads(res.text)

    company_list = [x['kor_isu_nm'] for x in data['DS1']]
    text = ', '.join(company_list) if company_list else '해당없음'
    return text

def get_투자위험종목(date):
    """
    투자위험종목 조회

    Args:
        date (str): 조회할 날짜. 형식: 'YYYYMMDD'.

    Returns:
        str: 투자위험종목.
    """
        
    url = "https://moc.krx.co.kr/contents/MOC/99/MOC99000001.jspx"

    payload = {
        "market_gubun": "ALL",
        "fromDate": date,
        "toDate": date,
        "pagePath": "/contents/SVL/M/03030200/MOC03030200.jsp",
        "code": "YmD69RMDlLdJjotrXv2qYDJKbIoJPL3rcbQNe5KZ0gaNzAj/pjFcZr/DzeNI0R76PsEoHQodui/1vJKOCFVpT+MwKfYjzRbM0Oit4+IttrphdOQWeKaynWsFPeb5zvmJm2lWk1xUbiHDmgZ/aniEgn4A3zcFVDOhfAMXg6H4Df2Dj7l7hV5e8Lq8Jh3vWyPORpSzuB4rQdTuRS5e1IwCXg==",
        "pageFirstCall": "Y"
    }

    res = requests.post(url, data=payload)
    data = json.loads(res.text)

    company_list = [x['kor_isu_nm'] for x in data['DS1']]
    text = ', '.join(company_list) if company_list else '해당없음'
    return text

def get_주요일정(week):
    """
    주요일정 조회

    Args:
        week (str): 'thisWeek' 또는 'nextWeek' 중 하나.

    Returns:
        str: 주요일정.
    """

    url = "https://kr.investing.com/economic-calendar/Service/getCalendarFilteredData"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36",
        "X-Requested-With": "XMLHttpRequest",
        "Accept": "text/html",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "keep-alive",
    }

    payload = {
        "timeZone": "88", # 서울 기준
        "country[]": ["5", "11"],  # 미국(5), 한국(11)
        "importance[]": ["3"], # 중요도 3
        "action": "filterEconomicCalendar", # 필수
        "currentTab": week
    }

    res = requests.post(url, headers=headers, data=payload)
    data = json.loads(res.text)['data']
    soup = BeautifulSoup(data, 'html.parser')
    rows = soup.find_all('tr')

    schedule = {}
    current_date = ''

    for tr in rows:
        tds = tr.find_all('td')
        
        if len(tds) == 1:
            current_date = f'{tds[0].text}' # YYYY년 MM월 DD일 X요일
            schedule[current_date] = []
            continue
            
        elif len(tds) == 8:
            currency = tds[1].text.strip()
            country = '한국' if currency == 'KRW' else '미국'

            title = tds[3].text.strip()
            actual = tds[4].text.strip()
            forecast = tds[5].text.strip()
            previous = tds[6].text.strip()

            excluded_keywords = ['신규 실업수당청구건수', '주택판매', '원유재고', '성명서', '연설', '증언', 'FOMC', '경제전망', '회의록', '기자회견', '필라델피아', 'JOLTS', 'CB', 'ISM', 'ADP', '시간당 임금']
            if any(keyword in title for keyword in excluded_keywords):
                continue

            if title[:2] == country: # 제목에 국가명이 포함되어있는 경우가 있음
                event_text = f'{title} (실제: {actual} // 예상치: {forecast} // 이전: {previous})'
            else:
                event_text = f'{country} {title} (실제: {actual} // 예상치: {forecast} // 이전: {previous})'
            
            schedule[current_date].append(event_text)

    previous_business_day = get_previous_business_day()

    result_text = ''
    for date_str, events in schedule.items():
        if not events:
            continue

        date = datetime.strptime(date_str[:-4], "%Y년 %m월 %d일").date()
        if date < previous_business_day:
            continue

        weekday = date_str[-3]

        result_text += f'\n{date.month}월 {date.day}일 ({weekday})\n'
        result_text += '\n\n'.join(events) + '\n'

    return result_text.strip() or "해당없음"

In [4]:
from datetime import datetime

now = datetime.now()
date = f'{now.year}{now.month:02d}{now.day:02d}'
weekdays = ['월', '화', '수', '목', '금', '토', '일']

주요일정 = get_주요일정('thisWeek')
if now.weekday() == 4: # 금요일
    다음주_주요일정 = get_주요일정('nextWeek')
    주요일정 += f'\n\n\n● 다음주 주요일정 (인베스팅닷컴)\n{다음주_주요일정}'

투자경고종목 = get_투자경고종목(date)
투자위험종목 = get_투자위험종목(date)



장전문자 = f'''
안녕하세요, SK증권 Passive영업팀입니다.
{now.year}년 {now.month}월 {now.day}일 ({weekdays[now.weekday()]}) 장시작 전 주요 증시&공시정보 안내드립니다.

● 해외시장 주요지표 변동 (한국기준 전영업일 15:30 ~ 당일 08:00)
- 미국 S&P500선물 : %
- 미국 나스닥 100선물 : %
- 미국 원달러환율(USD/KRW) : %


● 이번주 주요일정 (인베스팅닷컴)
{주요일정}


● 투자경고/위험 종목
- 경고
{투자경고종목}

- 위험
{투자위험종목}


● KOSPI200, KOSDAQ150 종목 주요 공시정보


● 최신 지수운영 공지(KOSPI200 및 KOSDAQ150)
해당없음

● 지수운영공지 종료 전 이벤트
해당없음
'''

print(장전문자)


안녕하세요, SK증권 Passive영업팀입니다.
2025년 10월 2일 (목) 장시작 전 주요 증시&공시정보 안내드립니다.

● 해외시장 주요지표 변동 (한국기준 전영업일 15:30 ~ 당일 08:00)
- 미국 S&P500선물 : %
- 미국 나스닥 100선물 : %
- 미국 원달러환율(USD/KRW) : %


● 이번주 주요일정 (인베스팅닷컴)
10월 1일 (수)
미국 제조업 구매관리자지수  (9월) (실제: 52.0 // 예상치: 52.0 // 이전: 53.0)

10월 2일 (목)
한국 소비자물가지수 (YoY)  (9월) (실제: 2.1% // 예상치: 2.0% // 이전: 1.7%)

한국 소비자물가지수 (MoM)  (9월) (실제: 0.5% // 예상치: 0.4% // 이전: -0.1%)

10월 3일 (금)
미국 서비스 구매관리자지수  (9월) (실제:  // 예상치: 53.9 // 이전: 53.9)


● 투자경고/위험 종목
- 경고
씨어스테크놀로지, 큐리옥스바이오시스템즈, 동양우, 라닉스, SAMG엔터, 일정실업, 아이티켐, 타스컴, 동일스틸럭스, 한스바이오메드, 비올, 이미지스, 크로넥스, 큐로홀딩스, 노을, 아퓨어스, 인카금융서비스, 제닉스로보틱스, 큐리언트, 프로티나, 원익홀딩스, 파멥신

- 위험
코오롱모빌리티그룹우, 코오롱모빌리티그룹


● KOSPI200, KOSDAQ150 종목 주요 공시정보


● 최신 지수운영 공지(KOSPI200 및 KOSDAQ150)
해당없음

● 지수운영공지 종료 전 이벤트
해당없음



In [6]:
today = datetime.today().date()
previous_business_day = get_previous_business_day()

for date in [today, previous_business_day]:
    print(f'================= {date} =================')
    df = get_공시(date)

    keywords = ["자기주식", "추가상장", "변경상장", "추가 상장", "변경 상장", "증자", "감자", "선택권", "CB전환", "소각", "처분", "취득", "배당", "주주명부폐쇄", "기준일", "투자경고", "투자위험", "투자유의", "정지", "단일가매매", "상장폐지", "신주인수권", "분할", "합병"]
    pattern = '|'.join(keywords)
    filtered_df = df[df['공시제목'].str.contains(pattern)]

    for _, row in filtered_df.iterrows():
        print(f"[{row['시간']}] {row['회사명']:<15} - {row['공시제목']} ({row['링크']})")
    print()

================= 2025-10-02 =================
[14:23] 유티아이            - 타법인주식및출자증권취득결정 (https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno=20251002000316&docno=&viewerhost=&viewerport=)
[13:52] 유티아이            - 자기주식처분결과보고서 (https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno=20251002000268&docno=&viewerhost=&viewerport=)
[09:30] 네이처셀            - 유형자산 취득결정(종속회사의 주요경영사항) (https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno=20251002000061&docno=&viewerhost=&viewerport=)

================= 2025-10-01 =================
[20:00] 녹십자홀딩스          - 단기과열종목(가격괴리율, 3거래일 단일가매매) 지정 연장(녹십자홀딩스2우) (https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno=20251001000709&docno=&viewerhost=&viewerport=)
[17:26] 인탑스             - 자기주식처분결과보고서 (https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno=20251001000921&docno=&viewerhost=&viewerport=)
[17:25] 카카오게임즈          - 유상증자결정(제3자배정) (https://kind.krx.co.kr/common/disclsviewer.do?method=s